# Variational Inference

- [Univariate (separate) models](#univariate-separate-models)
- [Bivariate Copula model](#bivariate-copula-model)

Libraries:

In [ ]:
import numpy as np
import torch
import pyro
from pyro import poutine
import pyro.distributions as dist
from pyro.infer import SVI, TraceEnum_ELBO
from pyro.infer.autoguide import AutoDelta
from pyro.optim import Adam,ClippedAdam
import pandas as pd
import tqdm
# from models.UnivariateHMM import UnivariateHMM
# from models.CopulaHMM import CopulaHMM
from utils.CopulaHelpers import *

Global variables:

In [ ]:
DATA_DIR="data/"
HIDDEN_STATES = 5
TRAINING_STEPS= 400

Import data:

In [ ]:
data = pd.read_csv(f"{DATA_DIR}hulls_df_matchday2_reduced.csv")
data = data.dropna()
# Convert the areas from m^2 to dam^2 for computational reasons
data["HomeHull"]=data["HomeHull"]/100
data["AwayHull"]=data["AwayHull"]/100
data.head()

## Univariate (separate) models

**NB: DA CAMBIARE IN UN BIVARIATO CON EMISSIONI INDIPENDENTI, COSI DA AVERE UNO STESSO STATO PER LE 2 SERIE STORICHE**

In [ ]:
def UnivariateHMM(sequence: torch.tensor, 
                  hidden_dim: int, 
                  include_prior: bool=True):
    '''
    Pyro Model for a Hidden Markov Model with a single univariate observation with Gamma emission distribution.
    Structure of the model taken from the Pyro documentation:
    https://pyro.ai/examples/hmm.html
    
    INPUTS:
    - sequence (torch.tensor): A 1-dimensional tensor of observations.
    - hidden_dim (int): The number of hidden states.
    - include_prior (bool): If True, include priors for the parameters of the model.
    '''
    length = len(sequence)
    with poutine.mask(mask=include_prior):
        # Transition probabilities
        probs_x = pyro.sample(
            "probs_x",
            dist.Dirichlet(0.9 * torch.eye(hidden_dim) + 0.1).to_event(),
        )
        # Emission probabilities (1-dimensional for the area)
        probs_alpha = pyro.sample(
            "probs_alpha",
            dist.Gamma(1.0, 1.0).expand([hidden_dim]).to_event(1)
        )

        probs_beta = pyro.sample(
            "probs_beta",
            dist.Gamma(1.0, 1.0).expand([hidden_dim]).to_event(1)
        )
    
    x = 0  # Initial hidden state
    for t in pyro.markov(range(length)):
        x = pyro.sample(
            f"x_{t}",
            dist.Categorical(probs_x[x]),
            infer={"enumerate": "parallel"},
        )
        pyro.sample(
            f"y_{t}",
            dist.Gamma(probs_alpha[x], probs_beta[x]),
            obs=sequence[t],
        )

## Bivariate copula model

In [ ]:
from models.CopulaHMM import CopulaHMM

In [ ]:
sequence_XY = torch.tensor(data[["HomeHull","AwayHull"]].values/100)
sequence_XY

In [ ]:
model = CopulaHMM(hidden_states=5)
model.fit(sequence_XY, training_steps=400)

In [ ]:
model.print_estimates()

In [ ]:
torch.save(model.posterior,"parameters/CopulaHMM_matchday2_5states_OldData.pt")